In [1]:
from IPython.display import Image, display

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent

from react_agent.src.agent_tools.sap_help_searcher import SapHelpSearcher
from react_agent.src.agent_tools.troubleshooting_searcher import TroubleshootingSearcher
from react_agent.src.agent_tools.codebase_searcher import CodebaseSearcher
from react_agent.src.agent_tools.source_code_retriever import SourceCodeRetriever

from react_agent.src.config.system_parameters import QAToolsServerSettings
from react_agent.src.config.system_parameters import CodingToolsServerSettings

from react_agent.src.util.llm_proxy import LLM_PROXY

In [2]:
# Wether to run MLFlow next to the agent
WITH_ML_FLOW = False

# True = Consume tooling from the local server
USE_MCP = False

USE_QA_TOOLS = True
USE_RCA_TOOLS = True

In [3]:
if WITH_ML_FLOW:
    import mlflow

    # poetry run mlflow server --host 127.0.0.1 --port 8080
    mlflow.set_tracking_uri(uri="http://localhost:8080")
    mlflow.set_experiment("Testing of email ReAct Agent")

    mlflow.langchain.autolog()

In [4]:
if USE_MCP:
    multiServerClientConfig = {}

    if USE_QA_TOOLS:
        QA_SERVER_SETTINGS = QAToolsServerSettings()

        multiServerClientConfig["QuestionAnsweringTools"] = {
                "url": f"http://{QA_SERVER_SETTINGS.host}:{QA_SERVER_SETTINGS.port}/{QA_SERVER_SETTINGS.transport}",
                "transport": QA_SERVER_SETTINGS.transport,
        }

    if USE_RCA_TOOLS:
        RCA_SERVER_SETTINGS = CodingToolsServerSettings()

        multiServerClientConfig["CodingTools"] = {
                "url": f"http://{RCA_SERVER_SETTINGS.host}:{RCA_SERVER_SETTINGS.port}/{RCA_SERVER_SETTINGS.transport}",
                "transport": RCA_SERVER_SETTINGS.transport,
        }
else:
    tool_list = []

    if USE_QA_TOOLS:
        tool_list.append(TroubleshootingSearcher())
        tool_list.append(SapHelpSearcher())
    if USE_RCA_TOOLS:
        tool_list.append(CodebaseSearcher())
        tool_list.append(SourceCodeRetriever())

In [5]:
mapping_question = "Explain how 'Payment Terms' is mapped. Start with 'map_invoice1'."
extensibility_question = "As a Public Cloud customer in Spain, can I extend an existing eDocument customer invoice Process?"
peppol_support = """We as Syntax are currently implementing SAP DRC and are following the official SAP help.
We are currently at the step “Integration of Peppol Exchange with SAP S/4HANA Cloud” and there at the creation of an S/4HANA Public Cloud communication user.
A client certificate should be uploaded for the communication user to be created, as Basic Auth (via user + password) is not supported.

The problem: We do not have this certificate. Where can I get this certificate? What format must it be in?
We are stuck at this step because we are missing the certificate mentioned and it is also not clear to us what kind of certificate is meant here and where to get it.
"""

In [6]:
query = mapping_question

if USE_MCP:
    async with MultiServerMCPClient( multiServerClientConfig ) as client:
        agent = ReActAgent(tool_list=client.get_tools())
        # print(agent.create_sys_prompt())
        # display(Image(agent.get_agent_graph()))
        await agent.arun_and_print_agent_stream(user_message=query)
else:
    agent = ReActAgent(tool_list=tool_list)
    # print(agent.create_sys_prompt())
    # display(Image(agent.get_agent_graph()))
    agent.run_and_print_agent_stream(user_message=query)

================================ Human Message =================================

Explain how 'Payment Terms' is mapped. Start with 'map_invoice1'.
================================== Ai Message ==================================
Tool Calls:
  codebase_memories_searcher (call_ie4nFKl97bFxzijMBnSv2SbR)
 Call ID: call_ie4nFKl97bFxzijMBnSv2SbR
  Args:
    query: map_invoice1 Payment Terms
================================= Tool Message =================================
Name: codebase_memories_searcher

Class: cl_edoc_map_ubl
Method Name: map_invoice1
Method Implementation:
METHOD map_invoice1.

  DATA:
    ls_target TYPE edo_ubl_invoice_request.

* Set source object
  ms_source = is_source.

* Set document category
  mv_doc_categ = 'INV'.                                     "2854416

* Initial mapping
  init_mapping( ).

* Fill Standard Business DocumentH eader
  ls_target-standard_business_document-standard_business_document_hea = fill_sbdh( ). "2805246, 2854416

* In case of resubmit fill

In [7]:
LLM_PROXY.print_usage()

Call Count: 3
Token Usage: {'input_tokens': 96164, 'output_tokens': 489, 'total_tokens': 96653}
